In [1]:
%%configure -f
{"jars":["/usr/hdp/current/hbase-client/lib/hbase-client.jar","/usr/hdp/current/hbase-client/lib/hbase-common.jar","/usr/hdp/current/hbase-client/lib/hbase-server.jar","/usr/hdp/current/hbase-client/lib/guava-12.0.1.jar","/usr/hdp/current/hbase-client/lib/hbase-protocol.jar","/usr/hdp/current/hbase-client/lib/htrace-core-3.1.0-incubating.jar","/home/adminDemo/shc-master/shc-master/target/hbase-spark-connector-1.0.0.jar"], "files":["/usr/hdp/current/spark-client/conf/hbase-site.xml"]}


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1464823593763_0032,spark,dead,,,
8,application_1464823593763_0037,spark,dead,,,
22,application_1464823593763_0049,spark,idle,Link,Link,


In [2]:
import org.apache.spark.sql.{SQLContext, _}

Creating SparkContext as 'sc'
Creating HiveContext as 'sqlContext'
import org.apache.spark.sql.{SQLContext, _}

In [3]:
import org.apache.spark.sql.execution.datasources.hbase._

import org.apache.spark.sql.execution.datasources.hbase._

In [4]:
import org.apache.spark.{SparkConf, SparkContext}

import org.apache.spark.{SparkConf, SparkContext}

In [5]:
case class HBaseRecordAirline(col0: String,Year: Int,Quarter: Int,Month: Int,DayofMonth: Int,DayOfWeek: Int,FlightDate: Int,UniqueCarrier: String,AirlineID: String)

defined class HBaseRecordAirline

In [6]:
object HBaseRecordAirlineTest {def apply(i: Int): HBaseRecordAirline = {val s = s"""row${"%03d".format(i)}""" 
                                                                        HBaseRecordAirline(s,i,i,i,i,i,i,s,s)}}

defined module HBaseRecordAirlineTest

In [7]:
val cat = s"""{
     |             |"table":{"namespace":"default", "name":"airdelaydata_scv_Test1"},
     |             |"rowkey":"key",
     |             |"columns":{
     |               |"col0":{"cf":"rowkey", "col":"key", "type":"string"},
     |               |"Year":{"cf":"Year", "col":"Year", "type":"int"},
     |               |"Quarter":{"cf":"Quarter", "col":"Quarter", "type":"int"},
     |               |"Month":{"cf":"Month", "col":"Month", "type":"int"},
     |               |"DayofMonth":{"cf":"DayofMonth", "col":"DayofMonth", "type":"int"},
     |               |"DayOfWeek":{"cf":"DayOfWeek", "col":"DayOfWeek", "type":"int"},
     |               |"FlightDate":{"cf":"FlightDate", "col":"FlightDate", "type":"int"},
     |               |"UniqueCarrier":{"cf":"UniqueCarrier", "col":"UniqueCarrier", "type":"string"},
     |               |"AirlineID":{"cf":"AirlineID", "col":"AirlineID", "type":"string"}
     |             |}
     |           |}""".stripMargin

cat: String = 
{
"table":{"namespace":"default", "name":"airdelaydata_scv_Test1"},
"rowkey":"key",
"columns":{
"col0":{"cf":"rowkey", "col":"key", "type":"string"},
"Year":{"cf":"Year", "col":"Year", "type":"int"},
"Quarter":{"cf":"Quarter", "col":"Quarter", "type":"int"},
"Month":{"cf":"Month", "col":"Month", "type":"int"},
"DayofMonth":{"cf":"DayofMonth", "col":"DayofMonth", "type":"int"},
"DayOfWeek":{"cf":"DayOfWeek", "col":"DayOfWeek", "type":"int"},
"FlightDate":{"cf":"FlightDate", "col":"FlightDate", "type":"int"},
"UniqueCarrier":{"cf":"UniqueCarrier", "col":"UniqueCarrier", "type":"string"},
"AirlineID":{"cf":"AirlineID", "col":"AirlineID", "type":"string"}
}
}

In [8]:
val sparkConf = new SparkConf().setAppName("HBaseTest")

sparkConf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@9414869

In [9]:
import sqlContext.implicits._

import sqlContext.implicits._

In [10]:
val data = (0 to 8).map { i =>HBaseRecordAirlineTest(i)}

data: scala.collection.immutable.IndexedSeq[HBaseRecordAirline] = Vector(HBaseRecordAirline(row000,0,0,0,0,0,0,row000,row000), HBaseRecordAirline(row001,1,1,1,1,1,1,row001,row001), HBaseRecordAirline(row002,2,2,2,2,2,2,row002,row002), HBaseRecordAirline(row003,3,3,3,3,3,3,row003,row003), HBaseRecordAirline(row004,4,4,4,4,4,4,row004,row004), HBaseRecordAirline(row005,5,5,5,5,5,5,row005,row005), HBaseRecordAirline(row006,6,6,6,6,6,6,row006,row006), HBaseRecordAirline(row007,7,7,7,7,7,7,row007,row007), HBaseRecordAirline(row008,8,8,8,8,8,8,row008,row008))

In [11]:
sc.parallelize(data).toDF.write.options(Map(HBaseTableCatalog.tableCatalog -> cat, HBaseTableCatalog.newTable -> "5")).format("org.apache.spark.sql.execution.datasources.hbase").save()

In [12]:
def withCatalog(cat: String): DataFrame = {
     |       sqlContext
     |         .read
     |         .options(Map(HBaseTableCatalog.tableCatalog->cat))
     |         .format("org.apache.spark.sql.execution.datasources.hbase")
     |         .load()
     |     }

withCatalog: (cat: String)org.apache.spark.sql.DataFrame

In [13]:
val df = withCatalog(cat)

df: org.apache.spark.sql.DataFrame = [UniqueCarrier: string, Month: int, col0: string, Quarter: int, FlightDate: int, AirlineID: string, DayOfWeek: int, DayofMonth: int, Year: int]

In [14]:
df.registerTempTable("table1")

In [15]:
val c = sqlContext.sql("select AirlineID from table1")

c: org.apache.spark.sql.DataFrame = [AirlineID: string]

In [16]:
c.show()

+---------+
|AirlineID|
+---------+
|   row000|
|   row001|
|   row002|
|   row003|
|   row004|
|   row005|
|   row006|
|   row007|
|   row008|
+---------+